In [ ]:
# default_exp core

# Core

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#librerias de rapid
import math
from ebm.models import RBM
from ebm.optimizers import SGD, Adam, outer_product
from ebm.samplers import ContrastiveDivergence as cd
from torch import cat, einsum, max, mm, randint, rand_like, sigmoid, sign,     \
                  sqrt, tanh, zeros_like
from torch.nn.functional import linear, dropout
from torch.nn import Parameter
from tqdm import tqdm

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Eloy")

'Hello Eloy!'

In [ ]:
assert say_hello("Jeremy")=="Hello Jeremy!"

In [ ]:
# -----------------------------------------------------------------------------
# Models
# -----------------------------------------------------------------------------
class RBM_pm(RBM):

    def __init__(self, n_visible=10, n_hidden=50, sampler=None, optimizer=None,
                 device=None, weights=None):
        '''Restricted Boltzmann machine with spin-like neurons (their allowed
           values are +1/-1) instead of binary (0/1) neurons, and with no
           biases.
           
        Arguments:
           
            :param n_visible: The number nodes in the visible layer
            :type n_visible: int
            :param n_hidden: The number nodes in the hidden layer
            :type n_hidden: int
            :param sampler: Method used to draw samples from the model
            :type sampler: :class:`ebm.samplers`
            :param optimizer: Optimizer used for parameter updates
            :type optimizer: :class:`ebm.optimizers`
            :param device: Device where to perform computations. None is CPU.
            :type device: torch.device
            :param W: Optional parameter to specify the weights of the RBM
            :type W: torch.nn.Parameter
            :param vbias: Optional parameter to specify the visible biases of
                          the RBM
            :type vbias: torch.nn.Parameter
            :param hbias: Optional parameter to specify the hidden biases of
                          the RBM
            :type hbias: torch.nn.Parameter
        '''
        super().__init__(n_visible, n_hidden, sampler, optimizer, device, weights)
    
    def free_energy(self, v):
        '''Computes the free energy for a given state of the visible layer.

        Arguments:

            :param v: The state of the visible layer of the RBM
            :type v: torch.Tensor

            :returns: torch.Tensor
        '''
        
        wx_b = linear(v, self.weights, self.hbias)
        
        # Fancy (and overflow-resistant) way of computing log(2cosh(x))
        a = max(wx_b, -wx_b)
        hidden_term = (a + ((-wx_b - a).exp() + (wx_b - a).exp()).log()).sum(1)
        return -hidden_term

    def train(self, input_data):
        '''Trains the RBM.

        Arguments:

            :param input_data: Batch of training points
            :type input_data: torch.utils.data.DataLoader
        '''
        for batch in tqdm(input_data, desc=('Epoch ' +
                                            str(self.optimizer.epoch + 1))):
            
            sample_data = batch.float()
            
            vpos = sample_data
            vneg = self.sampler.get_negative_sample(vpos, self.weights,
                                                   self.vbias, self.hbias)
            W_update, _, _ = \
                             self.optimizer.get_updates(vpos, vneg,
                                                self.weights, self.vbias, self.hbias)
            self.weights += W_update
             
        self.optimizer.epoch += 1

In [ ]:
class RA_RBM(RBM_pm):

    def __init__(self, n_visible=100, n_hidden=50, K=50,
                 optimizer=None, device=None, xi=None):
        '''RBM where the weights are computed through the method of Restricted
           Axon. The weights are computed from low-energy patterns, and the
           parameters to be optimized is the patterns themselves
        
        Arguments:
            
            :param n_visible: The number nodes in the visible layer
            :type n_visible: int
            :param n_hidden: The number nodes in the hidden layer
            :type n_hidden: int
            :param K: The number of patterns from which the weights are computed
            :type K: int
            :param sampler: Method used to draw samples from the model
            :type sampler: :class:`samplers`
            :param optimizer: Optimizer used for parameter updates
            :type optimizer: :class:`optimizers`
            :param device: Device where to perform computations. None is CPU.
            :type device: torch.device
            :param xi: Optional parameter to specify the initial patterns
            :type xi: torch.nn.Parameter
            :param vbias: Optional parameter to specify the visible biases of
                          the RBM
            :type vbias: torch.nn.Parameter
            :param hbias: Optional parameter to specify the hidden biases of
                          the RBM
            :type hbias: torch.nn.Parameter'''
        super().__init__(n_visible, n_hidden, 'None', optimizer, device)
        self.K         = K
        self.n_visible = n_visible
        self.n_hidden  = n_hidden
        
        if xi is not None:
            self.xi = xi
        else:
            self.xi = Parameter((2 * randint(0,
                                             2,
                                        (self.K, self.n_hidden + self.n_visible)
                                             ) - 1).float().to(self.device))
        for param in self.parameters():
            param.requires_grad = False  

        self.get_params()
            
    def get_params(self):
        '''Computes the weight matrix of the RBM from the patterns'''
        vis  = self.xi[:, :self.n_visible]
        hidd = self.xi[:, self.n_visible:]
        self.weights.data = (outer_product(hidd, vis).sum(0)).to(self.device)
        self.weights.data /= math.sqrt(self.K)
        
    def train(self, input_data):
        '''Trains the RBM.

        Arguments:

            :param input_data: Batch of training points
            :type input_data: torch.utils.data.DataLoader
        '''
        
        for batch in tqdm(input_data, desc=('Epoch ' +
                                            str(self.optimizer.epoch + 1))):

            sample_data = batch.float()

            vpos = sample_data
            # Get negative phase from the patterns
            vneg = sign(self.xi[:, :vpos.shape[1]])

            xi_update = self.optimizer.get_updates(vpos, vneg, self.xi)
            self.xi += xi_update

            self.get_params()
        
        # Renormalize after the training epoch has concluded
        self.xi.data = sign(self.xi)
        self.get_params()
        self.optimizer.epoch += 1    

In [ ]:
# -----------------------------------------------------------------------------
# Optimizers
# -----------------------------------------------------------------------------
class SGD_xi(SGD):
    
    def __init__(self, learning_rate, momentum=0, weight_decay=0):
        '''Update the value of the pattern units via Stochastic Gradient Descent
        
        Arguments:

            :param learning_rate: Learning rate
            :type learning_rate: float
            :param weight_decay: Weight decay parameter, to prevent overfitting
            :type weight_decay: float
            :param momentum: Momentum parameter, for improved learning
            :type momentum: float
        '''
        super().__init__(learning_rate, momentum, weight_decay)

    def get_params(self, xi, n_visible):
        '''Computes the neuron connections (axons) of the RBM from the patterns
        using the Hebbian rule
        
        Arguments:

            :param xi: Patterns from which to compute the weights
            :type xi: torch.Tensor
            :param n_visible: Number of visible neurons in the model
            :type n_visible: int
        '''
        vis = xi[:, :n_visible]
        hidd = xi[:, n_visible:]
        W = (mm(hidd.t(), vis) / math.sqrt(xi.shape[0])).to(xi.device)
        return W

    def get_updates(self, vpos, vneg, xi):
        '''Obtain the parameter updates
        
        Arguments:

            :param vpos: Batch of samples from the training set
            :type vpos: torch.Tensor
            :param vneg: Batch of samples drawn from the model
            :type vneg: torch.Tensor
            :param xi: Patterns from which to compute the weights
            :type xi: torch.Tensor
        '''
        if self.first_call:
            self.n_vis      = vpos.shape[1]
            self.pos_batch  = vpos.shape[0]
            self.neg_batch  = vneg.shape[0]
            self.K          = xi.shape[0]
            self.xi_update  = zeros_like(xi)
            self.first_call = False

        self.xi_update *= self.momentum
        self.xi_update -= self.learning_rate * self.weight_decay * xi
        
        W = self.get_params(xi, self.n_vis)
        xi_vis = xi[:, :self.n_vis]
        xi_hid = xi[:, self.n_vis:]
        deltaxi_v = (einsum('bj,bk->kj', (vpos, mm(tanh(mm(vpos, W.t())), xi_hid.t()))) / self.pos_batch
                   - einsum('bj,bk->kj', (vneg, mm(tanh(mm(vneg, W.t())), xi_hid.t()))) / self.neg_batch) / math.sqrt(self.K)
        deltaxi_h = (einsum('bj,kj,ba->ka',(vpos, xi_vis, tanh(mm(vpos, W.t())))) / self.pos_batch
                   - einsum('bj,kj,ba->ka',(vneg, xi_vis, tanh(mm(vneg, W.t())))) / self.neg_batch) / math.sqrt(self.K)
        deltaxi = cat([deltaxi_v, deltaxi_h], 1)
        self.xi_update.data += self.learning_rate * deltaxi

        return self.xi_update